<a href="https://colab.research.google.com/github/gactyxc/NDCI-mGMM/blob/main/NDCI_mGMM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initialize

In [ ]:
! pip install earthengine-api
import ee
ee.Authenticate()
ee.Initialize(project='crops-mapping-gaoyuan')
!pip install geemap
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
# global variable defination
year = 2020
startDoy = 180
endDoy = 210
roi = ee.Geometry.Rectangle(
    coords=[124.0, 44.2, 125.0, 45.2], proj='EPSG:4326', geodesic=False
)
process_dir = '/content/drive/MyDrive/process'
NDCI_samplesFilename = 'testRegion_cropSamples_VI'
probabilityMap_fpath = 'testRegion_probabilityMap'

In [ ]:
if not os.path.exists(process_dir):
  os.makedirs(process_dir)
  print("Folder created successfully.")
else:
  print("Folder already exists.")
ESA_landmap = ee.ImageCollection("ESA/WorldCover/v100")
ESA_croplandMask = ESA_landmap.first().eq(40)
# filter the sample use cropland mask
roi_croplandMask = ESA_croplandMask.clip(roi)
roi_croplandMask = roi_croplandMask.updateMask(roi_croplandMask)

# part-1: data preprocession

In [ ]:
# common functions
# function to get the sentinel-2 image collection based on the study data range and study area ----*/
# function to remove cloud
# function to exclude bad data at scene edges
def maskEdges(s2_img):
    return s2_img.updateMask(
        s2_img.select('B8A').mask().updateMask(s2_img.select('B9').mask()))

# Function to mask clouds in Sentinel-2 imagery.
def maskClouds(img):
    max_cloud_probabiltly = 50
    clouds = ee.Image(img.get('cloud_mask')).select('probability')
    isNotCloud = clouds.lt(max_cloud_probabiltly)
    return img.updateMask(isNotCloud)

def sentinel2_collection(start_data, end_data, roi):
    s2Sr = ee.ImageCollection("COPERNICUS/S2_HARMONIZED")
    s2Clouds = ee.ImageCollection("COPERNICUS/S2_CLOUD_PROBABILITY")

    # define the filter constraints
    criteria = ee.Filter.And(ee.Filter.geometry(roi), ee.Filter.date(start_data, end_data))

    # sentinel-2 data collection
    sentinel2_bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12']
    new_bands = ['B', 'G', 'R', 'RE1', 'RE2', 'RE3', 'NIR', 'RE4', 'SWIR1', 'SWIR2']

    # Filter input collections by desired data range and region.
    s2Sr = s2Sr.filter(criteria).map(maskEdges)
    s2Clouds = s2Clouds.filter(criteria)

    # Join S2 SR with cloud probability dataset to add cloud mask.
    s2SrWithCloudMask = ee.Join.saveFirst('cloud_mask').apply(**{
      "primary": s2Sr,
      "secondary": s2Clouds,
      "condition": ee.Filter.equals(**{"leftField": "system:index", "rightField":"system:index"})
      })

    # collect the images without cloud
    s2CloudMasked = ee.ImageCollection(s2SrWithCloudMask).map(maskClouds).select(sentinel2_bands, new_bands)
    return s2CloudMasked

# function to add all related VIs
# add NDCI, a normalized difference composite index for maize identification
def addNDCI(img):
    NDCI = img.expression('NDCI = 2500 * (10000-SWIR1 - G) / (7.5 * RE1 - SWIR1 + 20000)', {
        'RE1': img.select('RE1'),
        'G': img.select('G'),
        'SWIR1': img.select('SWIR1')
    })
    img = img.addBands(NDCI)
    img = img.toInt16()
    return img

# add LSWI
def addLSWI(image):
    LSWI = image.expression('LSWI = 1000 * (NIR - SWIR1) / (NIR + SWIR1)', {
        'SWIR1': image.select('SWIR1'),
        'NIR': image.select('NIR')
    })
    return image.addBands(LSWI)

# add EVI
def addEVI(image):
    EVI = image.expression('EVI = 2500 * (NIR - R) / (NIR + 6 * R - 7.5 * B + 10000)', {
        'NIR': image.select('NIR'),
        'R': image.select('R'),
        'B': image.select('B')
    })
    return image.addBands(EVI)

# part-2: get NDCI map

In [ ]:
# year: the target identification year
# startDoy: the start Doy of optimal identification window
# endDoy: the end Doy of optimal identification window
# roi: target region of study
# output value of this function is the time series s2sr images in a given time period
def get_s2sr_images(year, startDoy, endDoy, roi):
    # define the start and end time of identification
    startDate = ee.Date.fromYMD(year, 1, 1).advance(startDoy, 'day')
    endDate = ee.Date.fromYMD(year, 1, 1).advance(endDoy, 'day')

    # define the image collection
    s2SR_imgCol = sentinel2_collection(ee.Date.fromYMD(year, 1, 1),
                                       ee.Date.fromYMD(year, 12, 31), roi)

    # Create a date range list with a specified 10-day interval，use millis as unit
    dates = ee.List.sequence(startDate.millis(), endDate.millis(), 1000 * 60 * 60 * 24 * 10)

    # function to resample time resolution of image collection to 10 day
    def resampleTo10Days(date):
        currentDate = ee.Date(date)
        endDate = currentDate.advance(10, 'day')
        summarizedImageCol = s2SR_imgCol.filterDate(currentDate, endDate)
        summarizedImage = summarizedImageCol.median()
        summarizedImage = summarizedImage.set('system:time_start', date)
        return summarizedImage

    # Apply the time resampling function using map()
    resampledImages = ee.ImageCollection(dates.map(resampleTo10Days))
    resampledImages = resampledImages.map(addLSWI)
    resampledImages = resampledImages.map(addEVI)
    resampledImages = resampledImages.map(addNDCI)
    return resampledImages

In [ ]:
# year: the target identification year
# startDoy: the start Doy of optimal identification window
# endDoy: the end Doy of optimal identification window
# roi: target region of study
# output value of this function is the time series NDCI images in a given time period
def get_NDCI_map(resampledImages):
    # get the BSMI index and the remove the outlier pixels as 0
    def NDCI_mask(image):
      LSWI = image.select('LSWI')
      EVI = image.select('EVI')
      mask = EVI.lte(0.35).And(LSWI.add(ee.Image.constant(0.05)).gte(EVI))
      valid_mask = mask.Not()
      NDCI = image.select('NDCI')
      NDCI = NDCI.multiply(valid_mask).rename('NDCI_mask')
      NDCI = NDCI.toInt16()
      image = image.addBands(NDCI)
      return image.select('NDCI_mask')
    NDCI_Images = resampledImages.map(NDCI_mask)
    return NDCI_Images

# part-3: mGMM construction (GEE part)

In [ ]:
# ------ step-1. get the random sample in each 1°×1° grid
# this function use the ESA landMap as cropland mask
# roi: target region of study
# sampleSize: the random sample size in each grid, the default size is 0.1% of the number of pixels in each grid
# output value of this function is random samples in a given grid and given size
def get_random_sample(roi, sampleSize=None):

    if sampleSize is None:
        sampleSize = 124000

    randomPoints = ee.FeatureCollection.randomPoints(
        region=roi, points=sampleSize, seed=1234, maxError=1
        )

    def mask_points(point):
      isInsideMask = roi_croplandMask.reduceRegion(
          reducer=ee.Reducer.first(),
          geometry=point.geometry(),
          scale=10,
          maxPixels=1
      ).getNumber('Map')
      return point.set('inside_mask', isInsideMask)

    maskedPoints = randomPoints.map(mask_points)
    finalPoints = maskedPoints.filter(ee.Filter.eq('inside_mask', 1))
    return finalPoints

In [ ]:
#------ step-2. extract the NDCI value of each image for each random point
# imgCol: the extracted time series images
# pts: extraction points
# this funtion runs to derive the image values of given samples
def extract_points_value(imgCol, pts):
  ft = ee.FeatureCollection(ee.List([]))

  def fill(img, ini):
    date = ee.Date(img.date()).format()
    inift = ee.FeatureCollection(ini)
    ft2 = img.sampleRegions(
        collection = pts,
        properties = ['ID'], # Properties to include from points
        scale = 10
    )
    ft3 = ft2.map(lambda f: f.set('date', date))
    return inift.merge(ft3)
  newft = ee.FeatureCollection(imgCol.iterate(fill, ft))
  task = ee.batch.Export.table.toDrive(
      collection=newft,
      description = NDCI_samplesFilename,
      folder = 'process',
      fileFormat = 'CSV'
  )
  task.start()

In [ ]:
#%% main procedure
# part-1: get_NDCI_samples
imgCol = get_s2sr_images(year, startDoy, endDoy, roi)
NDCI_imgCol = get_NDCI_map(imgCol)
pts = get_random_sample(roi)
extract_points_value(NDCI_imgCol, pts)

# part-3: mGMM construction (python part)

In [ ]:
!pip install astropy

In [ ]:
# ---- step-1: separate the sample data by DOY
import os
import pandas as pd
import numpy as np
from astropy.time import Time

def separate_samples(file_path, doy_file_dir):
  f = open(file_path, 'r', encoding="utf-8")
  primary_data = pd.read_csv(f)
  data = primary_data.drop(['system:index','.geo'], axis=1)
  # get all DOY
  date = data['date'].values.tolist()
  data_time = Time(date,format='isot', scale='utc')
  start_time = Time(f'{year}-01-01', format='isot', scale='utc')
  data_doy = data_time.jd-start_time.jd
  data['doy'] = data_doy
  doy_list = np.unique(data_doy)
  if not os.path.exists(doy_file_dir):
    os.mkdir(doy_file_dir)
    for doy in doy_list:
      doy_str = str(int(doy))
      out_filename = doy_file_dir +'/'+ doy_str + '.csv'
      temp_data = data.loc[(data.doy == doy),:]
      out_data = temp_data['NDCI_mask']
      out_data.to_csv(out_filename, mode='a', index=False, header=False)
  print('sepatrate sample data down.')

In [ ]:
! pip install scikit-learn
!pip install scipy

In [ ]:
# ---- step-2: multi-temporal GMM construction
from scipy.stats import multivariate_normal
from sklearn.mixture import GaussianMixture

# function to removed the outlier samples based on the IQR
def remove_outlier_samples_IQR(sampled_pixels):
    Q1 = np.quantile(sampled_pixels,0.25)
    Q3 = np.quantile(sampled_pixels,0.75)
    IQR = Q3 - Q1
    min_range = Q1 - 1.5 * IQR
    max_range = Q3 + 1.5 * IQR
    filtered_pixels = sampled_pixels[(sampled_pixels >= min_range) & (sampled_pixels <= max_range)]
    return filtered_pixels
# determine whether the value is within the given interval
def if_in_range(x_arr,range_left,range_right):
    root = np.nan
    for x in x_arr:
        if (x > range_left and x < range_right):
            root = x
    return root
# function to calculate the OLR of the Gaussian Mixture Model
def calculate_OLR(mu,cov,pi):
    # get the intersection point of two guassian curve
    sigma1 = np.sqrt(cov[0])
    mu1 = mu[0]
    pi1 = pi[0]
    sigma2 = np.sqrt(cov[1])
    mu2 = mu[1]
    pi2 = pi[1]
    a = sigma1 ** 2 - sigma2 ** 2
    b = 2 * (sigma2 ** 2 * mu1 - sigma1 ** 2 * mu2)
    c = sigma1 ** 2 * mu2 ** 2 - sigma2 ** 2 * mu1 ** 2 - 2 *sigma1 ** 2 * sigma2 ** 2 * np.log((sigma1 * pi2)/ (sigma2 * pi1))
    p = np.poly1d([a[0][0], b[0][0], c[0][0]])
    g1 = multivariate_normal(mean=mu[0], cov=cov[0])
    g2 = multivariate_normal(mean=mu[1], cov=cov[1])
    p1 = pi[0]*g1.pdf(mu[0])
    p2 = pi[1]*g2.pdf(mu[1])
    p_subMax = min(p1,p2)
    intersections_x = p.r
    if mu[0] < mu[1]:
        root = if_in_range(intersections_x,mu[0],mu[1]) # intersection_x
    else:
        root = if_in_range(intersections_x,mu[1],mu[0]) # intersection_x
    if (~np.isnan(root)):
        p_saddle = pi[0]*g1.pdf(root) # intersection_y
        OLR = p_saddle / p_subMax # get the overlap rate (OLR)
    else:
        OLR = 1
    return OLR

def Multi_GMM_construction(samples_dir):
    samples_list = os.listdir(samples_dir)
    OLR_df=pd.DataFrame()
    for file in samples_list:
        doy = file.split('.')[0]
        inputfile = samples_dir +'/'+ file
        f = open(inputfile, 'r', encoding="utf-8")
        data = pd.read_csv(f)
        all_data = data.values.reshape(-1,1)
        data = all_data[all_data != 0].reshape(-1,1)
        X_train = remove_outlier_samples_IQR(data) # remove the outlier samples using IQR method
        gmm = GaussianMixture(n_components=2)  # constructe the GMM model with 2 cluster
        gmm.fit(X_train.reshape(-1,1))
        means = gmm.means_
        covs = gmm.covariances_
        weights = gmm.weights_
        OLR = calculate_OLR(means,covs,weights)
        #columns = ['DOY','mean_0','mean_1','cov_0','cov_1','weight_0','weight_1','OLR']
        if means[0][0] < means[1][0]:
            parameters = np.array([means[0][0],means[1][0],covs[0][0][0],covs[1][0][0],weights[0],weights[1],OLR])
        else:
            parameters = np.array([means[1][0],means[0][0],covs[1][0][0],covs[0][0][0],weights[1],weights[0],OLR])
        OLR_df[doy] = parameters
    OLRs = OLR_df.loc[6]
    print('all doy calculation down!')
    return OLR_df

In [ ]:
# step-3: get the GMM variables of each DOY based on the NDCI_samplesFilename
# check whether get_NDCI_samples task run out
import os
import time
samples_file_path = process_dir + '/' + NDCI_samplesFilename + '.csv'
doy_file_dir = process_dir + '/doy_samples'

while True:
  if os.path.exists(samples_file_path):
    print("Task run out!")
    with open(samples_file_path,'r') as file:
      # get the GMM variables of each DOY based on the NDCI_samplesFilename
      separate_samples(samples_file_path,doy_file_dir)
      GMM_variables = Multi_GMM_construction(doy_file_dir)
      print(GMM_variables)
    break;
  else:
    print("File not found. Waiting for 30 seconds...")
    time.sleep(30) # wait 30 seconds for next checking

# part-4: NDCI-mGMM implementation in GEE

In [ ]:
# define the mean, cov, weight and OLR
means_0 = GMM_variables.iloc[0,:].values
means_1 = GMM_variables.iloc[1,:].values
covs_0 = GMM_variables.iloc[2,:].values
covs_1 = GMM_variables.iloc[3,:].values
weights_0 = GMM_variables.iloc[4,:].values
weights_1 = GMM_variables.iloc[5,:].values
OLRs = GMM_variables.iloc[6,:].values

# define the imageCollection for GMM implementation
imgCol_list = imgCol.toList(imgCol.size())
n = imgCol_list.size().getInfo()
index_list = list(range(0, n))

# function to get the GMM probability of a given single image
def get_probability(index):
  image = ee.Image(imgCol_list.get(index))
  # mask by the ESA cropland mask
  image = image.updateMask(roi_croplandMask)
  # get the pdf value of each cluster based on the GMM
  pdf_1 = image.expression(
      '(wi / (sqrt(2 * 3.14159265359 * cov))) * exp(-((x - mean) * (x - mean) / (2 * cov)))', {
        'x': image.select('NDCI'),
        'mean': means_0[index],
        'cov': covs_0[index],
        'wi': weights_0[index]
    })
  pdf_2 = image.expression(
      '(wi / (sqrt(2 * 3.14159265359 * cov))) * exp(-((x - mean) * (x - mean) / (2 * cov)))', {
        'x': image.select('NDCI'),
        'mean': means_1[index],
        'cov': covs_1[index],
        'wi': weights_1[index]
    })
  probability = pdf_2.divide(pdf_1.add(pdf_2)).rename('maize_prop')

  # get the mask of outlier NDCI pixel rule
  NDCI_mask = ((image.select('EVI').lte(0.35)).And
             ((image.select('LSWI').add(ee.Image.constant(0.05))).gte(image.select('EVI')))).Not().rename('NDCI_mask')

  image = image.addBands(NDCI_mask)

  # Mask the probability to 0 with the mask
  probability = probability.updateMask(NDCI_mask)
  # Get the probability weight of each image based on the OLR
  OLR = OLRs[index]
  OLR_wi = ee.Image.constant(1 - OLR).rename('OLR_1')
  OLR_wi = OLR_wi.where(NDCI_mask.eq(0), 1)
  OLR_wi = OLR_wi.toFloat()

  image = image.addBands(probability)
  image = image.addBands(OLR_wi)
  return image

# get maize probability of each single image
prob_Collection = ee.ImageCollection.fromImages([get_probability(index) for index in index_list])
print('prob_Collection bandnames',prob_Collection.first().bandNames().getInfo())

sum_OLRwi = prob_Collection.select('OLR_1').reduce(ee.Reducer.sum())
prob_list = prob_Collection.toList(prob_Collection.size())
final_MaizeProb = ee.Image.constant(0)

for doy_num in index_list:
    index_image = ee.Image(prob_list.get(doy_num))
    BSMI_mask = index_image.select('NDCI_mask')
    probability = index_image.select('maize_prop')
    probability = probability.unmask(0)
    OLR_1 = index_image.select('OLR_1')
    prob_weight = OLR_1.divide(sum_OLRwi)
    MaizeProb = probability.multiply(prob_weight)
    MaizeProb = MaizeProb.unmask(0)
    final_MaizeProb = final_MaizeProb.add(MaizeProb)

final_MaizeProb = final_MaizeProb.multiply(1000).toInt16()

# export the probability map to drive
task = ee.batch.Export.image.toDrive(
    image=final_MaizeProb.clip(roi),
    description = probabilityMap_fpath,
    folder = 'process',
    fileNamePrefix = probabilityMap_fpath,
    region = roi,
    scale = 10,
    crs = "EPSG:4326",
    maxPixels = 1e13
)
task.start()

# main procedure

In [ ]:
import geemap

m = geemap.Map()
m.set_center(124.5, 44.7, 7)
m.add_layer(roi, {'color': 'yellow'}, 'Region')
m.add_basemap('SATELLITE')
m.add_layer(final_MaizeProb.clip(roi),{'min': 0,'max': 1000,'palette': ['gray', 'orange']},'final_maizeMap')
m